In [1]:
!pip install pyqlib

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import qlib
qlib.__version__

'0.9.6'

In [9]:
import pickle as pkl
import pandas as pd

with open(r'D:\金工组实习\科创版股票list.pkl', "rb") as f:
    object = pkl.load(f, encoding='utf-8')
    
df = pd.DataFrame(object)

df.to_csv(r'D:\金工组实习\科创版股票list.csv', index=False)

In [2]:
import pandas as pd
print(pd.__version__)

2.2.3


In [ ]:
!pip uninstall pandas

In [3]:
import numpy as np
print(np.__version__)

1.26.4


## 初始化运行环境

In [ ]:
#初始化数据环境
import qlib
qlib.init(provider_uri = "./dataset/qlib_sh_data/sh_data")

## 获取交易日期和全部股票代码

调用qlib.data 模块可读取原始数据。例如qlib.data.calendar 命令可读取指定时间区间内
交易日期；qlib.data.instruments 命令可定义股票池，参数market=’all’代表选取全部个股
构成股票池；qlib.data.list_instruments 命令可以展示指定时间区间内的股票池，时间区间
的意义在于，股票池可能会随时间动态变化，如指数成分股票池。

In [ ]:
from qlib.data import D

#获取2024年1-11月交易日期
trade_date = D.calendar(start_time='2024-01-01', end_time='2024-11-30', freq='day')
#展示前5个交易日期
print(trade_date[:5])

In [ ]:
#例: 获取2020年7-11月全部港股代码
instruments = D.instruments(market='all')
stock_list = D.list_instruments(instruments=instruments,
                                start_time='2020-07-01',
                                end_time='2020-11-30',
                                as_list=True)
#展示后5个股票代码
print(stock_list[-5:])

## 获取指定股票指定日期指定字段数据

调用qlib.data.features 模块可以获取指定股票指定日期指定字段数据，例如获取腾讯控股（0700 HK）在2020年1月初至11 月底的日频后复权收盘价和成交量。

In [ ]:
features_df = D.features(instruments=['HK0700'], fields=['$close','$volume'],
                         start_time='2020-01-01', end_time='2020-11-30',freq='day')
print(features_df.head())

## 自定义股票池

采用qlib.data.filter 模块自定义股票池。首先，使用 qlib.data.filter.NameDFilter
命令进行股票名称静态筛选，参数 name_rule_re 为纳入股票代码的正则表达式，如
HK[0-9!]表示以 HK开头，后续为数字或感叹号的股票代码，感叹号代表目前已退市股票。  
其次，使用 qlib.data.filter.ExpressionDFilter 命令进行股票因子表达式的动态筛选，参数
rule_expression 为入选的因子表达式，如$close>=1 代表收盘价应大于等于1元。随后，
通过qlib.data.instruments 命令的参数 filter_pipe，将两个筛选条件组装到一起。 

In [ ]:
from qlib.data.filter import NameDFilter, ExpressionDFilter
#静态Filter:除恒生指数(HKHSI)外的港股
nameDFilter = NameDFilter(name_rule_re='HK[0-9!]')
#动态Filter:后复权价格大于等于1元
expressionDFilter = ExpressionDFilter(rule_expression='$close >= 1')
#获取港股2020年7-11月满足条件的港股代码
instruments = D.instruments(market='all',filter_pipe=[nameDFliter,expressionDFilter])
stock_list = D.list_instruments(instruments=instruments,
                                start_time='2020-07-01',
                                end_time='2020-11-30',
                                as_list=True)
#展示后5个股票代码
print(stock_list[-5:])

## Alpha158 因子库 

Qlib提供Alpha158 和 Alpha360 两类量价因子库，用户也可根据需要自定义因子库

生成Alpha158因子调用qlib.contrib.data.handler 模块下的 Alpha158 类，具体命令为：  
from qlib.contrib.data.handler import Alpha158  
h = Alpha158(**data_handler_config)同。

### 生成Alpha158特征（当期因子）和标签（下期收益）

In [ ]:
from qlib.contrib.data.handler import Alpha158

#设置日期 股票池等参数
data_handler_config = {
    "start_time":"2020-01-01",
    "end_time":"2020-11-30",
    "fit_start_time":"2020-01-01",
    "fit_end_time":"2020-06-30",
    "instruments":instruments,
}

h = Alpha158(**data_handler_config)

#获取列名(因子名称)
print(h.get_cols())

### 获取Alpha158因子库标签（下期收益）

In [ ]:
#获取标签(收益)
Alpha158_df_label = h.fetch(col_set="label")
print(Alpha158_df_label)

### 获取Alpha158因子库特征（当期因子）

In [ ]:
#获取特征(因子)
Alpha158_df_feature = h.fetch(col_set="feature")
print(Alpha158_df_feature)

## LightGBM 选股策略构建 

### 导入Qlib模块代码  (和原文略有出入,有些写法更新了)

In [4]:
import time
import numpy as np
import pandas as pd 

import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.contrib.strategy import TopkDropoutStrategy
from qlib.contrib.evaluate import risk_analysis, backtest_daily
from qlib.utils import exists_qlib_data, init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord

### 定义股票池和基准指数代码

In [ ]:
market =  instruments
benchmark = "HKHSI"

### 因子生成参数data_handler_config和模型训练参数task设置

In [ ]:
###################################
# train model
###################################
'''
定义因子生成参数data_handler_config
data_handler_config相当于配置文件，字典类型，
用来定义完整数据起止日期（start_time和end_time），
拟合数据起止日期（fit_start_time和fit_end_time），
股票池（instruments）等。
拟合数据起止日期区间应为完整数据起止日期数据的子集
'''
data_handler_config = {
        "start_time": "2020-01-01",
        "end_time": "2020-11-30",
        "fit_start_time": "2020-01-01",
        "fit_end_time": "2020-06-30",
        "instruments": market,
}

'''
定义模型训练参数task
分为model和dataset
'''

# 第一项model为AI模型参数，
# 必须包含class（AI模型名称）和module_path（AI模型所在路径）两个子键；
# kwargs为model的可选子键，通过kwargs设置指定AI模型的超参数

task = {
    "model": {
        "class": "LGBModel",#A模型名称,此处为 LightGBM
        "module_path": "qlib.contrib.model.gbdt",#A模型所在路径
        "kwargs": {#LightGBM模型超参数
            "loss": "mse",#損失函数,此处为均方误差
            "colsample_bytree": 0.8879,#列采样比例
            "learning_rate": 0.0421,#学习率
            "subsample": 0.8789,#行采样比例
            "lambda_l1": 205.6999,#L1正则化惩罚系数
            "lambda_l2": 580.9768,#L2正则化惩罚系数
            "max_depth": 8,#最大树深
            "num_leaves": 210,#最大叶子节点数
            "num_threads": 20,#最大并行线程数
        },
    },
    
# 第二项dataset为数据集参数，必须包含class（数据集名称）和
# module_path（数据集所在路径）两个子键；kwargs为dataset的可选子键，
# 通过kwargs设置指定数据集的参数

    "dataset": {
        "class": "DatasetH",#数据集名称
        "module_path": "qlib.data.dataset",#数据集所在路径
        "kwargs": {#Dataset模型参数
            "handler": {#因子库参数,处理器
                "class": "Alpha158",#因子库名称,继承自DataHandlerLP
                "module_path": "qlib.contrib.data.handler",#因子库路径
                "kwargs": data_handler_config,#Apha158因子库参数
            },
            "segments": {#时间划分参数
                "train": ("2018-01-01", "2019-12-31"),#训练集时间区间
                "valid": ("2020-01-01", "2020-06-30"),#验证集时间区间
                "test": ("2020-07-01", "2021-01-22"),#测试集时间区间
            },
        },
    },
}

# model initiaiton, 实例化模型对象
model = init_instance_by_config(task["model"])
#实例化因子库数据集,从基础行情数据计算出的包含所有特征因子和标签值的数据集
dataset = init_instance_by_config(task["dataset"])

### 模型训练

In [ ]:
# start exp to train model
'''
正式进行训练
例: 读入数据及数据预处理时间开销约为54.120秒，训练模型时间开销约为118秒。
LightGBM模型迭代的实质是参数优化，
当验证集损失连续50轮未降低时停止迭代
例: 此处迭代71次后结束训练
''' 
t_start = time.time()

#qlib.workflow.start开启训练；R是流管理器
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    
    #model.fit拟合模型；
    model.fit(dataset)
    
    #qlib.workflow.save_objects保存模型；
    R.save_objects(trained_model=model)
    
    #qlib.workflow.get_recorder().id获取“实验”（即模型训练）记录的编号
    rid = R.get_recorder().id

t_end = time.time()
print('train model - Time count: %.3fs'%(t_end-t_start))

### 选股策略回测

In [ ]:
###################################
# prediction, backtest & analysis
###################################
'''
接下来设置策略回测参数port_analysis_config，
该参数为字典类型，又可以分为strategy和backtest两个子键。

第一项strategy为策略参数，
例如此处使用TopkDropout策略，
每日等权持有topk=50只股票，
同时每日卖出持仓股票中最新预测收益最低的n_drop=5只股票，
买入未持仓股票中最新预测收益最高的n_drop=5只股票。

第二项backtest为策略回测参数，
用于设置涨跌停限制、起始资金、业绩比较基准、成交价格、交易费率等信息

'''
port_analysis_config = {
    "strategy": {#策略参数
        "class": "TopkDropoutStrategy",#策略名称
        "module_path": "qlib.contrib.strategy.strategy",#策略所在路径
        "kwargs": {#TopkDropout策略参数
            "topk": 5,#每日持仓股票数
            "n_drop": 2,#每日换仓股票数
        },
    },
    "backtest": {#回测参数
        "verbose": False,#是否实时显示回测信息
        "limit_threshold": 0.095,#涨跌停限制,港股不设涨跌停
        "account": 100000,#起始资金
        "benchmark": benchmark,#业绩比较基准,此处为上证300
        "deal_price": "close",#成交价格,此处为收盘价
        "open_cost": 0.0005,#开仓交易费率
        "close_cost": 0.0015,#平仓交易费率
        "min_cost": 1000,#最低交易费用
    },
}

# backtest and analysis
'''
调用qlib.workflow模块正式进行回测，依次执行如下命令：
qlib.workflow.record_temp.SignalRecord初始化调仓信号；
qlib.workflow.record_temp.PortAnaRecord初始化回测及绩效分析；
'''

#qlib.workflow.start开启回测；
with R.start(experiment_name="backtest_analysis"):
    #qlib.workflow.get_recorder获取此前模型训练“实验”记录；
    recorder = R.get_recorder(rid, experiment_name="train_model")
    
    #recorder.load_object读取模型；
    model = recorder.load_object("trained_model")

    #qlib.workflow.get_recorder初始化回测“实验”记录；
    recorder = R.get_recorder()  
    ba_rid = recorder.id
    
    #回传给recorder
    sr = SignalRecord(model, dataset, recorder)
    #生成预测结果，利用recorder记录结果
    sr.generate()

    # 在测试集上执行回测
    #创建组合分析记录器，其中实验记录器把预测值带进来，并记录最终回测结果
    par = PortAnaRecord(recorder, port_analysis_config)
    par.generate()#par.generate生成回测及绩效分析结果

t_end = time.time()
print('backtest and analysis - Time count: %.3fs'%(t_end-t_start))

### 回测和绩效分析结果展示

In [ ]:
'''
完成策略回测后，调用qlib.contrib.report模块展示回测和绩效分析结果。
展示前首先执行qlib.workflow.get_recorder获取回测“实验”记录，
相关结果均储存为pkl格式，
执行recorder.load_object读取
预测结果pred.pkl、
回测报告report_normal.pkl、
仓位情况positions_normal.pkl
和持仓分析port_analysis.pkl
'''
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(ba_rid, experiment_name="backtest_analysis")
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime')
report_normal_df = recorder.load_object("portfolio_analysis/report_normal.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis.pkl")

### AI模型预测结果IC和Rank IC

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

#展示AI模型预测个股收益的IC和Rank IC值
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)